In [ ]:
import pandas as pd
import numpy as np
import jieba

In [ ]:
from keras.preprocessing import sequence
from keras.optimizers import SGD,RMSprop,Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU

In [ ]:
neg=pd.read_excel('neg.xls',header=None,index=None)
pos = pd.read_excel ('pos.xls',header = None,index = None)

In [ ]:
print(pos.index)

In [ ]:
print(pos.head())

In [ ]:
pos['mark']=1
neg['mark']=0
pn =pd.concat([pos,neg],ignore_index=True)
neglen=len(neg)
poslen=len(pos)
cw = lambda x: list(jieba.cut(x))
pn['words']= pn[0].apply(cw)
print(pn[:20])
print(pn['word'][0:20])

In [8]:
comment = pd.read_excel('sum.xls')
comment = comment[comment['rateContent'].notnull()]
comment['words'] =comment['rateContent'].apply(cw)
d2v_train = pd.concat([pn['words'],comment['words']],ignore_index = True)
w=[]
for i in d2v_train:
    w.extend(i)
print(w[:20])

['做', '父母', '一定', '要', '有', '刘墉', '这样', '的', '心态', '，', '不断', '地', '学习', '，', '不断', '地', '进步', '，', '不断', '地']


In [9]:
dict = pd.DataFrame(pd.Series(w).value_counts())
#print(dict[:20])
print(dict.values)
print(dict.index)

[[131885]
 [ 85960]
 [ 49352]
 ..., 
 [     1]
 [     1]
 [     1]]
Index(['，', '的', '。', '了', '是', '很', ' ', '我', '！', ',',
       ...
       '1118', '这六星', '傣', '尿性', '替人', '遮遮掩掩', '一鼓酸', '挺美', '变变', '称科健'],
      dtype='object', length=52964)


In [10]:
dict['id']= list(range(1,len(dict)+1))
print(dict['id'])
get_sent = lambda x: list(dict['id'][x])

，           1
的           2
。           3
了           4
是           5
很           6
            7
我           8
！           9
,          10
也          11
好          12
在          13
不          14
还          15
都          16
就          17
有          18
不错         19
.          20
没有         21
这          22
和          23
、          24
买          25
酒店         26
看          27
安装         28
人          29
说          30
        ...  
恶性事故    52935
用脑      52936
12L     52937
出出进进    52938
邻居们     52939
利用软件    52940
京东方     52941
轰炸      52942
789     52943
结认      52944
污斑      52945
查不多     52946
插地      52947
北陵      52948
彪       52949
进口设备    52950
叮当声     52951
全差评     52952
提别      52953
钻孔      52954
1118    52955
这六星     52956
傣       52957
尿性      52958
替人      52959
遮遮掩掩    52960
一鼓酸     52961
挺美      52962
变变      52963
称科健     52964
Name: id, Length: 52964, dtype: int64


In [11]:
pn['sent'] = pn['words'].apply(get_sent) #速度太慢，求出现的排名
maxlen = 50

In [12]:
print('Pad sequences( samples x time)')

Pad sequences( samples x time)


In [13]:
pn['sent'] = list(sequence.pad_sequences(pn['sent'],maxlen = maxlen))

In [ ]:
x = np.array(list(pn['sent']))[::2]#训练集
y = np.array(list(pn['mark']))[::2]
xt = np.array(list(pn['sent']))[1::2] #测试集
yt = np.array(list(pn['mark']))[1::2]
xa = np.array(list(pn['sent'])) #全集
ya = np.array(list(pn['mark']))

In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(len(dict) + 1, 256))
model.add(LSTM(128))  # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(xa, ya, batch_size=1, epochs=1 )  # 训练时间为若干个小时

classes = model.predict_classes(xa)
acc = np_utils.accuracy(classes, yt)

print('Test accuracy:',acc)

Build model...
Epoch 1/1
  565/21105 [..............................] - ETA: 9677s - loss: 0.6773 - acc: 0.5611